In [57]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px
import numpy as np
import re
import plotly.graph_objs as go
from datetime import datetime
import math

In [2]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

## Mobility Data and COVID Data Import and Clean

In [3]:
fips = pd.read_excel(r'data/fips_codes.xlsx',dtype={'fips':str})
full_df = pd.read_excel(r'data/mobility_report_US.xlsx')

In [4]:
covid = pd.read_csv(r'data/covid_confirmed.csv',dtype={'countyFIPS':str})

In [5]:
date_replace= {}
for date in covid.columns[4:]:
    if re.search('\d\/\d\d\/\d\d', date):
        date_replace[date] = datetime.strptime(date,'%m/%d/%y').strftime('%Y-%m-%d')
    else:
        date_replace[date] = datetime.strptime(date,'%m %d %y').strftime('%Y-%m-%d')
covid.rename(columns=date_replace,inplace=True)
dates= list(date_replace.values())

In [6]:
full_df_copy = full_df.copy()

In [7]:
full_df_copy.county = full_df.county.str.replace(' County', '').str.replace(' Parish','')

In [8]:
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

In [9]:
full_df_copy.state = full_df_copy.state.map(us_state_abbrev).fillna(full_df_copy.state)
full_final = pd.merge(full_df_copy,fips[['fips','county','state']],how='left',left_on=['state','county'],right_on=['state','county'])
final_by_state = full_final[(full_final.state != 'Total') & (full_final.county == 'Total')]
final_by_state.state = final_by_state.replace({'District of Columbia':'DC'})
full_final = full_final[(full_final.state != 'Total') & (full_final.county != 'Total')]

C:\Users\alexc\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [10]:
full_final[full_final.fips.isna()]

,state,county,date,retail,grocery and pharmacy,parks,transit stations,workplaces,residential,fips


In [11]:
full_final['text'] = full_final.county + ', ' + full_final.state
full_final.head()

,state,county,date,retail,grocery and pharmacy,parks,transit stations,workplaces,residential,fips,text
202,AL,Autauga,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN,01001,"Autauga, AL"
203,AL,Autauga,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN,01001,"Autauga, AL"
204,AL,Autauga,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0,01001,"Autauga, AL"
205,AL,Autauga,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0,01001,"Autauga, AL"
206,AL,Autauga,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0,01001,"Autauga, AL"


## ACS Data Import and Clean

In [12]:
acs = pd.read_csv(r'data/2018_ACS.csv')
state_acs = pd.read_csv(r'data/ACS_2018_states.csv')
acs.drop(columns=['id'],inplace=True), state_acs.drop(columns=['id'], inplace=True);

C:\Users\alexc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (67,95,119,148,149,150,151,158,159,166,167,176,177,180,181,184,185,188,189,192,193,212,213,216,217,220,221,224,225,228,229,231,310,311,312,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,376,377,380,381,392,393,396,397,404,405,408,409,416,417,420,421,424,425,428,429,432,433,436,437) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
state_acs.rename(columns = {'Geographic Area Name':'state'},inplace=True)

In [14]:
no_moe = acs.columns.str.contains('Margin')
cols = acs.columns.tolist()
acs = acs[[cols[x] for x in range(len(cols)) if not no_moe[x]]]

no_moe = state_acs.columns.str.contains('Margin')
cols = state_acs.columns.tolist()
state_acs = state_acs[[cols[x] for x in range(len(cols)) if not no_moe[x]]]

In [15]:
county_state = acs['Geographic Area Name'].tolist()
county, state = [],[]
for i in county_state:
    pattern = '(.*), (.*)'
    a = re.search(pattern, i)
    county.append(a.group(1)),state.append(a.group(2))

In [16]:
for c in range(len(county)):
    county[c] = county[c].replace(' County','')
    county[c] = county[c].replace('.','')
    county[c] = county[c].replace(' Parish','')
    county[c] = county[c].replace(' Borough','')
    county[c] = county[c].replace('city','City')

In [17]:
acs['county'] = county
acs['state']= state
acs.state = acs.state.map(us_state_abbrev).fillna(acs.state)
acs_fips = pd.merge(acs,fips[['fips','county','state']],how='left',left_on=['state','county'],right_on=['state','county'])
acs_fips['text'] = acs_fips.county +', ' +acs_fips.state

## Means of ACS Data by State 

In [18]:
means = acs_fips.mean(numeric_only=True)
means_df = means.to_frame().T
means_df['state'] = ['USA']

In [19]:
for state in acs_fips.state.unique():
    state_df = acs_fips[acs_fips.state == state].mean(numeric_only=True).to_frame().T
    state_df['state'] = state
    means_df = means_df.append(state_df,ignore_index=True)
means_df.head()

,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families),Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families),Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!With own children of the householder under 18 years,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!With own children of the householder under 18 years,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family!!With own children of the householder under 18 years,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family!!With own children of the householder under 18 years,...,Percent Estimate!!ANCESTRY!!Total population!!Welsh,Estimate!!ANCESTRY!!Total population!!West Indian (excluding Hispanic origin groups),Percent Estimate!!ANCESTRY!!Total population!!West Indian (excluding Hispanic origin groups),Estimate!!COMPUTERS AND INTERNET USE!!Total households,Percent Estimate!!COMPUTERS AND INTERNET USE!!Total households,Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a computer,Percent Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a computer,Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription,Percent Estimate!!COMPUTERS AND INTERNET USE!!Total households!!With a broadband Internet subscription,state
0,38106.342457,38106.342457,25046.818269,66.555602,10641.342775,25.926862,18401.320178,50.825748,7237.563972,17.604042,...,0.591470,937.907384,0.235201,38106.342457,38106.342457,33826.422661,83.415691,30637.661044,72.690866,USA
1,27765.208955,27765.208955,18356.791045,66.150746,7273.925373,24.405970,13184.865672,47.146269,4711.716418,15.417910,...,0.320896,157.925373,0.156716,27765.208955,27765.208955,23153.671642,77.055224,20362.955224,65.762687,AL
2,8740.068966,8740.068966,5780.448276,65.175862,2744.689655,30.231034,4339.551724,45.075862,1907.000000,18.996552,...,0.724138,51.862069,0.124138,8740.068966,8740.068966,8158.551724,88.741379,7369.034483,75.655172,AK
3,168286.666667,168286.666667,109875.066667,67.013333,45582.866667,25.146667,80008.866667,48.366667,30006.266667,15.966667,...,0.553333,741.866667,0.120000,168286.666667,168286.666667,151270.866667,83.386667,137626.066667,71.726667,AZ
4,15362.333333,15362.333333,10197.546667,67.053333,4243.493333,25.446667,7451.560000,49.174667,2747.920000,16.205333,...,0.388000,47.613333,0.121333,15362.333333,15362.333333,12925.760000,79.258667,10728.466667,64.164000,AR


In [20]:
pop_col = 'Estimate!!RELATIONSHIP!!Population in households'

In [21]:
state_pop = state_acs[['state',pop_col]]
state_pop['state'] = state_pop.state.map(us_state_abbrev).fillna(state_pop.state)
state_pop.rename(columns={'Estimate!!RELATIONSHIP!!Population in households':'population'},inplace=True)
state_pop.state = state_pop.state.replace({'District of Columbia':'DC'})
state_pop.head()

C:\Users\alexc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\alexc\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,state,population
0,IL,12444301.0
1,IA,3057213.0
2,MO,5952011.0
3,NV,2996777.0
4,PA,12385066.0


In [22]:
test_state = final_by_state.copy()
cases_by_state = pd.DataFrame(columns=['cases','state'])
for state in covid.State.unique():
    state_cases = covid[covid.State == state][covid.columns[4:]].sum().to_frame().rename(columns={0:'cases'})
    state_cases['state'] = [state for _ in range(len(state_cases))]
    cases_by_state = cases_by_state.append(state_cases)
cases_by_state['date'] = cases_by_state.index

In [23]:
mob_case_state = pd.merge(test_state,cases_by_state,left_on=['date','state'],right_on=['date','state'])

In [24]:
usa_regions = {
    'WA' : 'North West','OR' : 'North West','ID' : 'North West','MT' : 'North West','WY' : 'North West',
    'CA' : 'West','NV' : 'West','AK' : 'West','HI' : 'West',
    'UT' : 'South West','NM' : 'South West','CO' : 'South West','AZ' : 'South West','TX' : 'South West','OK' : 'South West',
    'ND' : 'Mid-West','SD' : 'Mid-West','NE' : 'Mid-West','KS' : 'Mid-West','WI' : 'Mid-West','IA' : 'Mid-West','MO' : 'Mid-West',
    'MI' : 'Mid-West','IL' : 'Mid-West','IN' : 'Mid-West','KY' : 'Mid-West','OH' : 'Mid-West','MN' : 'Mid-West',
    'AR' : 'South East','LA' : 'South East','AL' : 'South East','MS' : 'South East','TN':'South East','GA':'South East','FL':'South East',
    'SC' : 'South East','NC' : 'South East',
    'VA' : 'Mid-Atlantic','WV' : 'Mid-Atlantic','PA' : 'Mid-Atlantic','MD' : 'Mid-Atlantic','DE' : 'Mid-Atlantic','NJ' : 'Mid-Atlantic',
    'NY' : 'Mid-Atlantic','DC' : 'Mid-Atlantic',
    'CT' : 'North East','RI' : 'North East','VT' : 'North East','NH' : 'North East','MA' : 'North East','ME' : 'North East'
    
}

In [25]:
mob_case_state['region'] = mob_case_state.state.map(usa_regions)
cases_by_state['region'] = cases_by_state.state.map(usa_regions)

In [26]:
covid.head()

,countyFIPS,County Name,State,stateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01001,Autauga County,AL,1,0,0,0,0,0,0,...,295,312,323,331,357,368,373,375,400,411
2,01003,Baldwin County,AL,1,0,0,0,0,0,0,...,331,343,353,361,364,383,389,392,401,413
3,01005,Barbour County,AL,1,0,0,0,0,0,0,...,208,214,221,226,234,238,245,251,263,266
4,01007,Bibb County,AL,1,0,0,0,0,0,0,...,89,93,97,100,104,111,116,118,121,126


In [27]:
proportion = pd.DataFrame(columns=['cases'])
for state in mob_case_state.state.unique():
    prop = (mob_case_state[mob_case_state.state == state].cases / state_pop[state_pop.state == state].population.sum()).to_frame()
    proportion = proportion.append(prop)

In [28]:
mob_case_state['proportion'] = proportion.cases*100

In [58]:
metric = 'grocery and pharmacy'
lower = mob_case_state[mob_case_state.date.isin(dates[55:])][metric].min()
upper = mob_case_state[mob_case_state.date.isin(dates[55:])][metric].max()

In [62]:
[int(math.ceil(upper / 10.0)) * 10, int(math.floor(lower/10.0))*10]

[40, -70]

In [66]:
def scatter_range(metric):
    if metric == 'parks':
        return [-100,250]
    lower = mob_case_state[mob_case_state.date.isin(dates[55:])][metric].min()
    upper = mob_case_state[mob_case_state.date.isin(dates[55:])][metric].max()
    return [int(math.floor(lower/10.0))*10,int(math.ceil(upper / 10.0)) * 10]

In [68]:
scatter_range('parks')

[-80, 360]

In [51]:
mob_case_state[mob_case_state.date.isin(dates[55:])]['grocery and pharmacy'].min(),mob_case_state[mob_case_state.date.isin(dates[55:])]['grocery and pharmacy'].max()

(-62.0, 34.0)

In [71]:
fig = px.scatter(mob_case_state[mob_case_state.date.isin(dates[55:])],
          x='parks', y='cases',animation_frame = 'date',animation_group = 'state',
           color = 'region', hover_name = 'state',size=mob_case_state[mob_case_state.date.isin(dates[55:])].proportion.tolist(),
          log_y = True, range_x = [-100,250],range_y = [1,400000])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.update_layout(
    #transition = {'duration' : 2000},
    title_text = 'Comparison of Mobility to Grocery Stores and COVID-19 Cases by Geographic Region',
    title_x = .5
    )
fig.show()

The `grocery and pharmacy` and the `parks` both are very interesting. 

`grocery and pharmacy` shows that the South East states and the Mid-West states are significantly less likely to be reducing the number of people who are going grocery shopping.

`parks` on the other hand shows that almost across the board the states most likely to be going to state and national parks are those who live in the Mid-West. --> what could the reason for this be?

`residential` is actually quite interesting as well. From Google: "The Residential category shows a change in duration—the other categories measure a change in total visitors. Because people already spend much of the day at places of residence (even on workdays), the capacity for change isn’t so large." So in that graphic we notice that the Mid-Atlantic and North East are seeing the highest percentage difference in people staying home, and we are approaching nearly 100% of the day spent there.

In [33]:
state = 'CA'
county = 'Alameda County'

In [39]:
fig = px.scatter(
    x=covid[(covid.State == state) & (covid['County Name'] == county)].columns[55:], 
    y=covid[(covid.State == state) & (covid['County Name'] == county)][covid.columns[55:]].iloc[0].tolist(),
    #animation_frame = 'date',animation_group = 'state',
    #color = 'region', #hover_name = 'State',#size=mob_case_state[mob_case_state.date.isin(dates[55:])].proportion.tolist(),
    #log_y = True,range_y = [1,400000]
    )
fig.update_layout(
    title_text = 'Trend of COVID-19 Cases in ' + county + ', ' + state
    )
fig.show()

In [ ]:
np.nanquantile(full_final[full_final.date.isin(dates[23:51])]['grocery and pharmacy'].tolist(),.15),np.nanquantile(full_final[full_final.date.isin(dates[23:51])]['grocery and pharmacy'].tolist(),.85)

In [43]:
files = {
    'grocery and pharmacy' : 'html/grocery_anim.html'
}

In [47]:
files['grocery and pharmacy']

'grocery_anim.html'

In [42]:
# Animated HTML better than what we had been making with the movies...
fig = px.choropleth(full_final[full_final.date.isin(dates[23:51])],
    locations='fips',
    geojson=counties,
    color = 'grocery and pharmacy',
    scope='usa',
    animation_frame = 'date',
    animation_group = 'fips',
    range_color = (-40,10)          
             )
fig.update_layout(
    title_text = 'Changing Movement to Grocery Stores',
    title_x = .5)
fig.write_html(r'html/grocery_anim.html')

# Test Figure
fig = go.Figure(data=go.Choropleth(
    locations = full_final.fips,
    z = full_final.parks,
    geojson = counties,
    zmin = -60, zmid = -35, zmax = 95,
    colorscale = 'Viridis',
    colorbar_title = 'difference<br>from baseline',
    colorbar_ticksuffix = '%',
    text = full_final[(full_final.date=='2020-03-20')].text
    ))

fig.update_layout(
    title={
        'text': 'March 20, 2020',
        'y':.1,
        'x':.1
    },
    geo_scope = 'usa'
    )

## Set of Graphics for March

In [45]:
#Retail
for date in dates[21:51]:
    datetime_obj = datetime.strptime(date,'%Y-%m-%d')
    fig = go.Figure(data=go.Choropleth(
        locations = full_final[full_final.date==date].fips,
        z = full_final[full_final.date==date].retail,
        geojson = counties,
        zmin = -60, zmid = 0, zmax = 15,
        colorscale = 'Viridis',
        colorbar_title = 'difference<br>from baseline',
        colorbar_ticksuffix = '%'
    ))
    fig.update_layout(
        title={
            'text': datetime_obj.strftime('%B')+' '+str(datetime_obj.day)+', '+str(datetime_obj.year),
            'y':.1,
            'x':.1
        },
        geo_scope = 'usa',
        height = 1000, width = 2000
    )
    #fig.write_html(r'html/retail_html/retail_'+date+'.html')
    #fig.write_image(r'png/retail_scrn/retail_'+date+'.png')

In [46]:
#Grocery And Pharmacy
for date in dates[21:51]:
    datetime_obj = datetime.strptime(date,'%Y-%m-%d')
    fig = go.Figure(data=go.Choropleth(
        locations = full_final[full_final.date==date].fips,
        z = full_final[full_final.date==date]['grocery and pharmacy'],
        geojson = counties,
        zmin = -25, zmid = 0, zmax = 10,
        colorscale = 'Viridis',
        colorbar_title = 'difference<br>from baseline',
        colorbar_ticksuffix = '%'
    ))
    fig.update_layout(
        title={
            'text': datetime_obj.strftime('%B')+' '+str(datetime_obj.day)+', '+str(datetime_obj.year),
            'y':.1,
            'x':.1
        },
        geo_scope = 'usa',
        height = 1000, width = 2000
    )
    #fig.write_html(r'html/grocery_html/grocery_'+date+'.html')
    #fig.write_image(r'png/grocery_scrn/grocery_'+date+'.png')

In [47]:
#Parks
for date in dates[21:51]:
    datetime_obj = datetime.strptime(date,'%Y-%m-%d')
    fig = go.Figure(data=go.Choropleth(
        locations = full_final[full_final.date==date].fips,
        z = full_final[full_final.date==date]['parks'],
        geojson = counties,
        zmin = -60, zmid = -35, zmax = 95,
        colorscale = 'Viridis',
        colorbar_title = 'difference<br>from baseline',
        colorbar_ticksuffix = '%'
    ))
    fig.update_layout(
        title={
            'text': datetime_obj.strftime('%B')+' '+str(datetime_obj.day)+', '+str(datetime_obj.year),
            'y':.1,
            'x':.1
        },
        geo_scope = 'usa',
        height = 1000, width = 2000
    )
    #fig.write_html(r'html/parks_html/parks'+date+'.html')
    #fig.write_image(r'png/parks_scrn/parks'+date+'.png')

In [48]:
#Transit Stations
for date in dates[21:51]:
    datetime_obj = datetime.strptime(date,'%Y-%m-%d')
    fig = go.Figure(data=go.Choropleth(
        locations = full_final[full_final.date==date].fips,
        z = full_final[full_final.date==date]['transit stations'],
        geojson = counties,
        zmin = -70, zmid = 0, zmax = 5,
        colorscale = 'Viridis',
        colorbar_title = 'difference<br>from baseline',
        colorbar_ticksuffix = '%'
    ))
    fig.update_layout(
        title={
            'text': datetime_obj.strftime('%B')+' '+str(datetime_obj.day)+', '+str(datetime_obj.year),
            'y':.1,
            'x':.1
        },
        geo_scope = 'usa',
        height = 1000, width = 2000
    )
    #fig.write_html(r'html/transit_html/transit'+date+'.html')
    #fig.write_image(r'png/transit_scrn/transit'+date+'.png')

In [50]:
#Workplaces
for date in dates[21:51]:
    datetime_obj = datetime.strptime(date,'%Y-%m-%d')
    fig = go.Figure(data=go.Choropleth(
        locations = full_final[full_final.date==date].fips,
        z = full_final[full_final.date==date]['workplaces'],
        geojson = counties,
        zmin = -60, zmid = 0, zmax = 5,
        colorscale = 'Viridis',
        colorbar_title = 'difference<br>from baseline',
        colorbar_ticksuffix = '%'
    ))
    fig.update_layout(
        title={
            'text': datetime_obj.strftime('%B')+' '+str(datetime_obj.day)+', '+str(datetime_obj.year),
            'y':.1,
            'x':.1
        },
        geo_scope = 'usa',
        height = 1000, width = 2000
    )
    #fig.write_html(r'html/workplaces_html/workplaces_'+date+'.html')
    #fig.write_image(r'png/workplaces_scrn/workplaces_'+date+'.png')